In [1]:
from langgraph.graph import END, START, StateGraph, MessagesState
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langgraph.checkpoint.memory import MemorySaver
import os


os.environ["OPENAI_API_KEY"] = ""

question = """
    Given an array of integers nums and an integer target, return indices of the two numbers such that they add up to target.
    You may assume that each input would have exactly one solution, and you may not use the same element twice.

    You can return the answer in any order.

    Input: nums = [2,7,11,15], target = 9
    Output: [0,1]
    Explanation: Because nums[0] + nums[1] == 9, we return [0, 1].
"""

expectedCode = """class Solution:
        def twoSum(self, nums: List[int], target: int) -> List[int]:
            hashmap = {}
            for i in range(len(nums)):
                hashmap[nums[i]] = i
            for i in range(len(nums)):
                complement = target - nums[i]
                if complement in hashmap and hashmap[complement] != i:
                    return [i, hashmap[complement]]

        """
expectedApproach = "A simple implementation uses two iterations. In the first iteration, we add each element's value as a key and its index as a value to the hash table. Then, in the second iteration, we check if each element's complement (target−nums[i]) exists in the hash table. If it does exist, we return current element's index and its complement's index. Beware that the complement must not be nums[i] itself!"

In [2]:
# Defining the interview state object

from pydantic import BaseModel, Field
from typing import List, Literal
from enum import Enum


class Phase(str, Enum):
    INTRODUCTION = 'introduction'
    QUESTION  = 'question'

class PhaseModel(BaseModel):
    phase : Literal['introduction', 'question']
    
class Dialogue(BaseModel):
    dialogue: str = Field(description="The dialogue")
    speaker: str = Field(description="The speaker of the dialogue")
    
class InterviewState(BaseModel):
    
    conversations: List[Dialogue] = Field(default_factory=list)
    question: str = question
    expectedApproach : str = expectedApproach
    expectedCode : str = expectedCode

    userCode : str = ""    
    phase : Phase = Phase.INTRODUCTION


class IntroductionResponse(BaseModel):
    dialogue : str
    

In [3]:
def readPrompt(filePath):
    content = open(filePath).read()
    return PromptTemplate.from_template(content)

def getModel():
    return  ChatOpenAI(
        model="gpt-4o-mini",
        temperature=0,
    )


In [4]:
#Defining nodes of the workflow

def controlNode(state : InterviewState):
    prompt = readPrompt("prompts\control\classifyConversation.txt")
    model = getModel()
    chain = prompt | model.with_structured_output(PhaseModel)
    response = chain.invoke({'conversations':state.conversations})
    state.phase  =  Phase(response.phase)
    return state.dict()
    

def introductionNode(state : InterviewState):

    prompt = readPrompt("prompts\introduction\introduction.txt")
    model = getModel()

    chain = prompt | model

    response = chain.invoke({'conversations':state.conversations})

    state.conversations.append(Dialogue(dialogue=str(response.content), speaker="INTERVIEWER"))
    
    return state.dict()


def describeQuestionNode(state : InterviewState):
    prompt = readPrompt("prompts\questions\describeQuestion.txt")
    model = getModel()

    chain = prompt | model

    response = chain.invoke({'conversations':state.conversations, 'question':state.question})

    state.conversations.append(Dialogue(dialogue=str(response.content), speaker="INTERVIEWER"))
    
    return state.dict()

In [5]:
from typing import Annotated, Literal, TypedDict
from langgraph.graph import END

def checkPhase(state: InterviewState) -> Literal["introductionNode", "describeQuestionNode",END]:
  
    if state.phase == Phase.INTRODUCTION :
        return "introductionNode"
    if state.phase == Phase.QUESTION:
        return "describeQuestionNode"
    
    return END

In [6]:
# Defining the graph

workflow = StateGraph(InterviewState)


workflow.add_node("controlNode", controlNode)
workflow.add_node("introductionNode", introductionNode)
workflow.add_node("describeQuestionNode", describeQuestionNode)

workflow.add_edge(START, "controlNode")
workflow.add_conditional_edges(
    "controlNode",
    checkPhase,
)
graph = workflow.compile()

In [7]:
memory : dict[str, InterviewState]= {'1' : InterviewState()}

In [8]:
from elevenlabs import VoiceSettings
from elevenlabs.client import ElevenLabs
import time

def read_text(text):
    client = ElevenLabs(
        api_key="",
    )
    response  = client.text_to_speech.convert(
        voice_id="cjVigY5qzO86Huf0OWal",
        optimize_streaming_latency="0",
        output_format="mp3_22050_32",
        text=text,
        voice_settings=VoiceSettings(
            stability=0.1,
            similarity_boost=0.3,
            style=0.2,
        ),
    )
    save_file_path = f"output.mp3"

    with open(save_file_path, "wb") as f:
        for chunk in response:
            if chunk:
                f.write(chunk)

    os.system("start output.mp3") 


def process_user_input(user_input):
    try:
        state : InterviewState = memory['1']
        state.conversations.append(Dialogue(dialogue=user_input, speaker="USER"))
        finalState = graph.invoke(state)
        memory['1'] = InterviewState(**finalState) 
        return memory['1']
    except Exception as e:
        print(e)

def chat():
    while True:
        user_input = input("")
        if user_input.lower() == 'exit':
            break
        response = process_user_input(user_input)
        text = (response.conversations[len(response.conversations)-1].dialogue)
        print(text)
        read_text(text)
        

c:\Users\vigne\OneDrive\Desktop\projects\smart-judge\scripts\.venv\lib\site-packages\pydantic\_internal\_config.py:341: UserWarning: Valid config keys have changed in V2:
* 'allow_population_by_field_name' has been renamed to 'populate_by_name'
* 'smart_union' has been removed
  warnings.warn(message, UserWarning)


In [9]:
chat()

"Hi, I'm a software engineer here, working on developing and optimizing our applications. Could you please share a bit about yourself?"
"That's great to hear! What specific projects or technologies are you currently working on at Amazon?"
"Great! Now that we’ve covered introductions, let’s dive into a data structures and algorithms question. 

Here’s the problem: You are given an array of integers called `nums` and an integer called `target`. Your task is to return the indices of the two numbers in the array that add up to the target. 

To clarify a few points: 

1. You can assume that there is exactly one solution for each input.
2. You cannot use the same element twice to form the sum.
3. The order of the indices in your output doesn’t matter.

For example, if you have the input `nums = [2, 7, 11, 15]` and `target = 9`, the output should be `[0, 1]` because `nums[0] + nums[1]` equals `9`.

Do you have any questions about the problem statement or any specific part of it that you’d lik